# AI Skills + Anthropic Claude

This notebook demonstrates how to use AI Skills with Anthropic's Claude models.

## Setup

In [ ]:
!pip install aiskills[anthropic,search] -q

## Configure API Key

Get your API key from [Anthropic Console](https://console.anthropic.com/).

In [ ]:
import os
from getpass import getpass

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass("Enter your Anthropic API key: ")

## Quick Start: 3 Lines of Code

In [ ]:
from aiskills.integrations import create_anthropic_client

client = create_anthropic_client()
response = client.chat("Help me debug a memory leak in Python")
print(response)

## How It Works

Claude's automatic tool use:
1. You send a message to Claude
2. Claude recognizes it needs a skill and makes a tool call
3. The tool executes locally (searches your skills library)
4. Results are automatically fed back to Claude
5. Claude generates the final response

## Multi-Turn Chat

In [ ]:
# Multi-turn conversation
messages = [
    {"role": "user", "content": "What skills do you have for testing?"}
]

response1 = client.chat_with_messages(messages)
print("Response 1:")
print(response1)
print("\n" + "="*50 + "\n")

# Continue the conversation
messages.append({"role": "assistant", "content": response1})
messages.append({"role": "user", "content": "Tell me more about the Python debugging one"})

response2 = client.chat_with_messages(messages)
print("Response 2:")
print(response2)

## Direct Skill Operations

You can also call skill operations directly:

In [ ]:
# List available skills
skills = client.list_skills()
print(f"Found {len(skills)} skills:\n")
for skill in skills[:5]:
    print(f"  - {skill['name']}: {skill['description'][:50]}...")

In [ ]:
# Search for skills
results = client.search_skills("authentication security")
print(f"Found {results.total} skills:\n")
for r in results.results:
    print(f"  - {r['name']}")

In [ ]:
# Use a skill directly
result = client.use_skill("help me with async programming in Python")
if result.success:
    print(f"Skill: {result.skill_name}")
    print(f"Tokens: {result.tokens_used}")
    print(f"\nPreview:\n{result.content[:500]}...")
else:
    print(f"Error: {result.error}")

## Using Different Models

In [ ]:
# Claude 3.5 Sonnet (default) - balanced
client_sonnet = create_anthropic_client(model="claude-sonnet-4-20250514")

# Claude 3 Haiku - faster, cheaper
client_haiku = create_anthropic_client(model="claude-3-haiku-20240307")

# Claude 3 Opus - most capable
client_opus = create_anthropic_client(model="claude-3-opus-20240229")

# Compare responses
question = "What's the best way to handle errors in Python?"

print("Haiku response:")
print(client_haiku.chat(question)[:300] + "...")

## Manual Tool Handling

For complete control over tool execution:

In [ ]:
from aiskills.integrations import create_anthropic_client

# Disable auto-execution
manual_client = create_anthropic_client(auto_execute=False)

# Get raw response
response = manual_client.get_completion_with_skills(
    messages=[{"role": "user", "content": "What skills do you have for Python?"}]
)

# Check for tool use
for block in response.content:
    if block.type == "tool_use":
        print(f"Tool: {block.name}")
        print(f"Input: {block.input}")
        
        # Execute manually
        result = manual_client.execute_tool(block.name, block.input)
        print(f"Result keys: {list(result.keys())}")
    elif block.type == "text":
        print(f"Text: {block.text[:200]}...")

## Available Tools

See what tools are available to Claude:

In [ ]:
from aiskills.integrations import get_anthropic_tools

tools = get_anthropic_tools()
print("Available tools for Claude:\n")
for tool in tools:
    print(f"Tool: {tool['name']}")
    print(f"  Description: {tool['description'][:80]}...")
    print(f"  Parameters: {list(tool['input_schema']['properties'].keys())}")
    print()

## Example: Code Review Assistant

In [ ]:
from aiskills.integrations import create_anthropic_client

client = create_anthropic_client()

code_to_review = '''
def fetch_all_users():
    users = []
    for i in range(10000):
        response = requests.get(f"https://api.example.com/users/{i}")
        if response.status_code == 200:
            users.append(response.json())
    return users
'''

response = client.chat(f"Review this code and suggest improvements:\n```python\n{code_to_review}\n```")
print(response)

## With Custom System Prompt

In [ ]:
response = client.chat(
    "How do I optimize database queries?",
    system_prompt="You are a database expert. Be concise and provide specific examples."
)
print(response)

## Next Steps

- Try different Claude models for various use cases
- Install more skills: `aiskills install <source>`
- Create your own skills: `aiskills init my-skill`
- Run the API server: `aiskills api serve`

For more information, see the [documentation](https://github.com/sergioc/ai-skills).